In [27]:
import numpy as np
import pandas as pd

In [28]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [29]:
file_path = "/content/drive/Shared drives/Hackathon/data/waiting_times.csv"
df = pd.read_csv(file_path)

In [30]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3509324 entries, 0 to 3509323
Data columns (total 14 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   WORK_DATE                 object 
 1   DEB_TIME                  object 
 2   DEB_TIME_HOUR             int64  
 3   FIN_TIME                  object 
 4   ENTITY_DESCRIPTION_SHORT  object 
 5   WAIT_TIME_MAX             int64  
 6   NB_UNITS                  float64
 7   GUEST_CARRIED             float64
 8   CAPACITY                  float64
 9   ADJUST_CAPACITY           float64
 10  OPEN_TIME                 int64  
 11  UP_TIME                   int64  
 12  DOWNTIME                  int64  
 13  NB_MAX_UNIT               float64
dtypes: float64(5), int64(5), object(4)
memory usage: 374.8+ MB
None


In [31]:
print(df.head())

    WORK_DATE                 DEB_TIME  DEB_TIME_HOUR  \
0  2018-01-01  2018-01-01 21:00:00.000             21   
1  2018-01-01  2018-01-01 19:30:00.000             19   
2  2018-01-01  2018-01-01 22:30:00.000             22   
3  2018-01-01  2018-01-01 12:45:00.000             12   
4  2018-01-01  2018-01-01 17:00:00.000             17   

                  FIN_TIME ENTITY_DESCRIPTION_SHORT  WAIT_TIME_MAX  NB_UNITS  \
0  2018-01-01 21:15:00.000           Roller Coaster              0       2.0   
1  2018-01-01 19:45:00.000              Bumper Cars              5      18.0   
2  2018-01-01 22:45:00.000              Rapids Ride              0       1.0   
3  2018-01-01 13:00:00.000              Crazy Dance              5       1.0   
4  2018-01-01 17:15:00.000                   Skyway              5      15.0   

   GUEST_CARRIED  CAPACITY  ADJUST_CAPACITY  OPEN_TIME  UP_TIME  DOWNTIME  \
0            0.0     0.000             0.00          0        0         0   
1          148.0   254

In [32]:
# Create a new column to check the sum
df["SUM_UP_DOWN"] = df["UP_TIME"] + df["DOWNTIME"]

# Check how many rows satisfy the condition
correct_count = (df["SUM_UP_DOWN"] == 15).sum()
incorrect_count = (df["SUM_UP_DOWN"] != 15).sum()

# Print results
print(f"Number of rows where UP_TIME + DOWNTIME == 15: {correct_count}")
print(f"Number of rows where UP_TIME + DOWNTIME != 15: {incorrect_count}")


Number of rows where UP_TIME + DOWNTIME == 15: 1833080
Number of rows where UP_TIME + DOWNTIME != 15: 1676244


In [33]:
### Step 0: Filter dataset for "PortAventura World" ###
# Define the list of attractions for PortAventura World
attr_PAW = [
    'Bumper Cars', 'Bungee Jump', 'Circus Train', 'Crazy Dance', 'Dizzy Dropper',
    'Drop Tower', 'Flying Coaster', 'Free Fall', 'Giant Wheel', 'Giga Coaster',
    'Go-Karts', 'Haunted House', 'Himalaya Ride', 'Inverted Coaster', 'Kiddie Coaster',
    'Merry Go Round', 'Oz Theatre', 'Rapids Ride', 'Roller Coaster', 'Spinning Coaster',
    'Spiral Slide', 'Superman Ride', 'Swing Ride', 'Vertical Drop', 'Water Ride', 'Zipline'
]

# Filter dataset to keep only PortAventura World attractions
df = df[df["ENTITY_DESCRIPTION_SHORT"].isin(attr_PAW)]

# Print unique values after filtering
unique_values = df["ENTITY_DESCRIPTION_SHORT"].unique()
print("Unique Attractions in Filtered Dataset:")
print(unique_values)

Unique Attractions in Filtered Dataset:
['Roller Coaster' 'Bumper Cars' 'Rapids Ride' 'Crazy Dance' 'Free Fall'
 'Swing Ride' 'Drop Tower' 'Spinning Coaster' 'Superman Ride'
 'Spiral Slide' 'Inverted Coaster' 'Water Ride' 'Oz Theatre'
 'Circus Train' 'Giant Wheel' 'Kiddie Coaster' 'Bungee Jump' 'Zipline'
 'Haunted House' 'Go-Karts' 'Dizzy Dropper' 'Merry Go Round'
 'Flying Coaster' 'Giga Coaster' 'Himalaya Ride' 'Vertical Drop']


In [34]:
### Step 1: Check for missing values ###
missing_counts = df.isnull().sum()
total_missing = missing_counts.sum()
print("Missing values per column before handling:\n", missing_counts)
print(f"Total missing values: {total_missing}")

Missing values per column before handling:
 WORK_DATE                   0
DEB_TIME                    0
DEB_TIME_HOUR               0
FIN_TIME                    0
ENTITY_DESCRIPTION_SHORT    0
WAIT_TIME_MAX               0
NB_UNITS                    0
GUEST_CARRIED               0
CAPACITY                    0
ADJUST_CAPACITY             0
OPEN_TIME                   0
UP_TIME                     0
DOWNTIME                    0
NB_MAX_UNIT                 0
SUM_UP_DOWN                 0
dtype: int64
Total missing values: 0


In [35]:
### Step 2: Check for missing dates ###
# Convert "USAGE_DATE" to datetime
date_col = "WORK_DATE"
df[date_col] = pd.to_datetime(df[date_col])

# Detect Missing Dates ###
# Generate the full expected date range
date_range = pd.date_range(start=df[date_col].min(), end=df[date_col].max())

# Find the missing dates
existing_dates = set(df[date_col])
missing_dates = sorted(set(date_range) - existing_dates)

# Print missing dates info
print(f"Total missing dates: {len(missing_dates)}")
if missing_dates:
    print(pd.DataFrame({"Missing Dates": missing_dates}).to_string(index=False))  # Print all missing dates

# Add Missing Dates to the Dataset ###
# Reindex DataFrame with full date range
#df = df.set_index("USAGE_DATE").reindex(date_range).reset_index()
#df.rename(columns={"index": "USAGE_DATE"}, inplace=True)

# Forward-fill or interpolate missing values
#df["FACILITY_NAME"] = df["FACILITY_NAME"].fillna(method="ffill")  # Ensure facility name is filled
#df["attendance"] = df["attendance"].interpolate(method="linear")  # Fill missing attendance values

Total missing dates: 0


In [36]:
### Step 3: negative value
# List of columns to check for negative values
columns_to_check = ["WAIT_TIME_MAX", "NB_UNITS", "GUEST_CARRIED", "CAPACITY", "ADJUST_CAPACITY", "OPEN_TIME",  "UP_TIME",  "DOWNTIME", "NB_MAX_UNIT"]

# Loop through each column and count negative values
for col in columns_to_check:
    negative_count = (df[col] < 0).sum()
    print(f"Number of negative values in {col}: {negative_count}")

Number of negative values in WAIT_TIME_MAX: 0
Number of negative values in NB_UNITS: 3
Number of negative values in GUEST_CARRIED: 209
Number of negative values in CAPACITY: 0
Number of negative values in ADJUST_CAPACITY: 0
Number of negative values in OPEN_TIME: 0
Number of negative values in UP_TIME: 65
Number of negative values in DOWNTIME: 0
Number of negative values in NB_MAX_UNIT: 0


In [37]:
# Filter rows where UP_TIME is negative
negative_up_time_df = df[df["DOWNTIME"] > 15]

# Display the filtered DataFrame
print("Rows with Negative UP_TIME:")
# Print all rows where UP_TIME is negative
print(negative_up_time_df.to_string(index=False))  # Prints all rows without truncation

Rows with Negative UP_TIME:
 WORK_DATE                DEB_TIME  DEB_TIME_HOUR                FIN_TIME ENTITY_DESCRIPTION_SHORT  WAIT_TIME_MAX  NB_UNITS  GUEST_CARRIED  CAPACITY  ADJUST_CAPACITY  OPEN_TIME  UP_TIME  DOWNTIME  NB_MAX_UNIT  SUM_UP_DOWN
2018-05-15 2018-05-15 10:30:00.000             10 2018-05-15 10:45:00.000           Flying Coaster              5      24.0            0.0     756.0              0.0         15      -15        30         24.0           15
2018-05-15 2018-05-15 10:45:00.000             10 2018-05-15 11:00:00.000           Flying Coaster              5      24.0            0.0     756.0              0.0         15      -15        30         24.0           15


In [38]:
# List of columns to apply forward fill for negative values
columns_to_fix = ["NB_UNITS", "GUEST_CARRIED"]

for col in columns_to_fix:
    # Replace negative values with NaN
    df.loc[df[col] < 0, col] = np.nan

    # Forward-fill NaN values (i.e., replacing negatives with last valid positive value)
    df[col] = df[col].fillna(method="ffill")

# Print to verify
print("Fixed NB_UNITS and GUEST_CARRIED:")
print(df[["NB_UNITS", "GUEST_CARRIED"]].head(10))  # Show first 10 rows

Fixed NB_UNITS and GUEST_CARRIED:
    NB_UNITS  GUEST_CARRIED
0        2.0         0.0000
1       18.0       148.0000
2        1.0         0.0000
3        1.0        46.0000
5        3.0         0.0000
7        2.0        51.0000
8       12.0        74.0000
10      16.0        41.9999
11       6.0       309.0000
14       2.0         0.0000


<ipython-input-38-8661fbe9b2c1>:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method="ffill")
<ipython-input-38-8661fbe9b2c1>:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method="ffill")


In [39]:
# Replace negative UP_TIME values with 0
df["UP_TIME"] = df["UP_TIME"].apply(lambda x: max(0, min(x, 15)))

# Ensure OPEN_TIME, UP_TIME, and DOWNTIME do not exceed 15
columns_to_cap = ["OPEN_TIME", "UP_TIME", "DOWNTIME"]

for col in columns_to_cap:
    df[col] = df[col].apply(lambda x: min(x, 15))  # Cap maximum at 15

# Print to verify changes
print("Updated values (first 10 rows):")
print(df[["WORK_DATE", "OPEN_TIME", "UP_TIME", "DOWNTIME"]].head(10))

Updated values (first 10 rows):
    WORK_DATE  OPEN_TIME  UP_TIME  DOWNTIME
0  2018-01-01          0        0         0
1  2018-01-01         15       15         0
2  2018-01-01          0        0         0
3  2018-01-01         15       15         0
5  2018-01-01          0        0         0
7  2018-01-01         15       15         0
8  2018-01-01         15       15         0
10 2018-01-01         15       15         0
11 2018-01-01         15       15         0
14 2018-01-01          0        0         0


In [40]:
# List of columns to check for negative values
columns_to_check = ["WAIT_TIME_MAX", "NB_UNITS", "GUEST_CARRIED", "CAPACITY", "ADJUST_CAPACITY", "OPEN_TIME",  "UP_TIME",  "DOWNTIME", "NB_MAX_UNIT"]

# Loop through each column and count negative values
for col in columns_to_check:
    negative_count = (df[col] < 0).sum()
    print(f"Number of negative values in {col}: {negative_count}")

Number of negative values in WAIT_TIME_MAX: 0
Number of negative values in NB_UNITS: 0
Number of negative values in GUEST_CARRIED: 0
Number of negative values in CAPACITY: 0
Number of negative values in ADJUST_CAPACITY: 0
Number of negative values in OPEN_TIME: 0
Number of negative values in UP_TIME: 0
Number of negative values in DOWNTIME: 0
Number of negative values in NB_MAX_UNIT: 0


In [41]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2369821 entries, 0 to 3509323
Data columns (total 15 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   WORK_DATE                 datetime64[ns]
 1   DEB_TIME                  object        
 2   DEB_TIME_HOUR             int64         
 3   FIN_TIME                  object        
 4   ENTITY_DESCRIPTION_SHORT  object        
 5   WAIT_TIME_MAX             int64         
 6   NB_UNITS                  float64       
 7   GUEST_CARRIED             float64       
 8   CAPACITY                  float64       
 9   ADJUST_CAPACITY           float64       
 10  OPEN_TIME                 int64         
 11  UP_TIME                   int64         
 12  DOWNTIME                  int64         
 13  NB_MAX_UNIT               float64       
 14  SUM_UP_DOWN               int64         
dtypes: datetime64[ns](1), float64(5), int64(6), object(3)
memory usage: 289.3+ MB
None


In [42]:
# Display the cleaned dataset
print("Cleaned Dataset:")
print(df.head())

Cleaned Dataset:
   WORK_DATE                 DEB_TIME  DEB_TIME_HOUR                 FIN_TIME  \
0 2018-01-01  2018-01-01 21:00:00.000             21  2018-01-01 21:15:00.000   
1 2018-01-01  2018-01-01 19:30:00.000             19  2018-01-01 19:45:00.000   
2 2018-01-01  2018-01-01 22:30:00.000             22  2018-01-01 22:45:00.000   
3 2018-01-01  2018-01-01 12:45:00.000             12  2018-01-01 13:00:00.000   
5 2018-01-01  2018-01-01 18:15:00.000             18  2018-01-01 18:30:00.000   

  ENTITY_DESCRIPTION_SHORT  WAIT_TIME_MAX  NB_UNITS  GUEST_CARRIED  CAPACITY  \
0           Roller Coaster              0       2.0            0.0     0.000   
1              Bumper Cars              5      18.0          148.0   254.749   
2              Rapids Ride              0       1.0            0.0     0.000   
3              Crazy Dance              5       1.0           46.0   250.001   
5                Free Fall             50       3.0            0.0     0.000   

   ADJUST_CAPAC

In [43]:
### Step 5: Export the cleaned dataset ###
cleaned_file_path = "/content/drive/Shared drives/Hackathon/data/cleaned_data/waiting_times_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved to: {cleaned_file_path}")

Cleaned dataset saved to: /content/drive/Shared drives/Hackathon/data/cleaned_data/cleaned_waiting_times.csv
